In [1]:
# Setup Distributed Client
autostart = True
if autostart:
    pass # just let it use the dask lab extension cluster
else:
    from dask.diagnostics import ProgressBar
    from dask.distributed import Client
    from dask.distributed import LocalCluster
    # client = Client('localhost:8787') # Connect to old dashboard?
    try:
        client = Client('tcp://localhost:8786', timeout='2s')
    except OSError:
        cluster = LocalCluster(scheduler_port=8786)
        client = Client(cluster)
        
client

Client Scheduler: tcp://127.0.0.1:38141 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 40 Memory: 134.93 GB


In [3]:
import sys
sys.path.insert(1,r"/export/data1/jbenjami/Code/Python/")
print(sys.path)
import personal

import numpy as np

from importlib import reload
import make_forcing

import modify_forcing
import xarray as xr
import os

In [5]:
os.getcwd()

'/export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/python'

In [14]:
# sites are at https://www.agci.org/sites/default/files/pdfs/lib/main/0805_1430_SBony.pdf (CFMIP3 at https://www.researchgate.net/figure/CFMIP-3-CMIP6-cfSites-locations-The-contours-give-an-indication-of-inter-model-spread-in_fig1_312935878)
model          = 'HadGEM2-A'
exper          = 'amip'
rip            = 'r1i1p1' # rip nomenclature, In the CMIP5 project, ensemble members are named in the rip-nomenclature, r for realization, i for initialisation and p for physics, followed by an integer, e.g. r1i1p1.
data_path      = '/export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/' # zhaoyi used /export/data1/zhaoyi/GCMForcedLES/cfmip2_output/'
ds             = None
    
forcing_path = data_path + model + '/forcing/'        # zhaoyi used '/export/data1/zhaoyi/GCMForcedLES/forcing/'
f_root = os.path.join(data_path, model, exper+'/')

ds = xr.open_mfdataset(f_root+'*'+'_cfSites_'+model+'_'+exper+'_'+rip+'_'+'*.nc', decode_cf=True, decode_times=True,combine='by_coords',data_vars="minimal", coords="minimal", compat="override",parallel=True)
ds = ds.sortby('time')
ds

<xarray.Dataset>
Dimensions:   (bnds: 2, lev: 38, site: 119, time: 144000)
Coordinates:
  * time      (time) object 2000-09-01 00:00:00 ... 2008-12-30 23:30:00
  * site      (site) int32 1 2 3 4 5 6 7 8 9 ... 112 113 114 115 116 117 118 119
  * lev       (lev) float64 20.0 80.0 180.0 ... 2.922e+04 3.291e+04 3.925e+04
    lat       (site) float32 dask.array<chunksize=(119,), meta=np.ndarray>
    lon       (site) float32 dask.array<chunksize=(119,), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    lev_bnds  (lev, bnds) float64 dask.array<chunksize=(38, 2), meta=np.ndarray>
    b         (lev) float64 dask.array<chunksize=(38,), meta=np.ndarray>
    orog      (site) float32 dask.array<chunksize=(119,), meta=np.ndarray>
    b_bnds    (lev, bnds) float64 dask.array<chunksize=(38, 2), meta=np.ndarray>
    cl        (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    cli       (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    clw       (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    hur       (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    hus       (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    pfull     (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    ta        (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    tnhusa    (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    tnta      (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    tntr      (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    ua        (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    va        (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    wap       (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    zg        (time, site, lev) float32 dask.array<chunksize=(17280, 119, 38), meta=np.ndarray>
    hfls      (time, site) float32 dask.array<chunksize=(17280, 119), meta=np.ndarray>
    hfss      (time, site) float32 dask.array<chunksize=(17280, 119), meta=np.ndarray>
    pr        (time, site) float32 dask.array<chunksize=(69120, 119), meta=np.ndarray>
    ps        (time, site) float32 dask.array<chunksize=(17280, 119), meta=np.ndarray>
    rsdt      (time, site) float32 dask.array<chunksize=(17280, 119), meta=np.ndarray>
    ts        (time, site) float32 dask.array<chunksize=(17280, 119), meta=np.ndarray>
Attributes:
    institution:            Met Office Hadley Centre, Fitzroy Road, Exeter, D...
    institute_id:           MOHC
    experiment_id:          amip
    source:                 HadGEM2-A (2009) atmosphere: HadGAM2 (N96L38);  l...
    model_id:               HadGEM2-A
    forcing:                GHG, Oz, SA, LU, Sl, Vl, BC, OC, (GHG = CO2, N2O,...
    parent_experiment_id:   N/A
    parent_experiment_rip:  N/A
    branch_time:            0.0
    contact:                chris.d.jones@metoffice.gov.uk,  mark.webb@metoff...
    history:                MOHC pp to CMOR/NetCDF convertor (version 1.10.4)...
    references:             Bellouin N. et al, (2007) Improved representation...
    initialization_method:  1
    physics_version:        1
    tracking_id:            0f5ad0f6-e54a-480b-a1f7-1674f2394457
    mo_runid:               akpcd
    product:                output
    experiment:             AMIP
    frequency:              subhr
    creation_date:          2011-12-18T02:38:09Z
    Conventions:            CF-1.4
    project_id:             CMIP5
    table_id:               Table cfSites (26 July 2011) 76cc5a96989ebb70010f...
    title:                  HadGEM2-A model output prepared for CMIP5 AMIP
    parent_experiment:      N/A
    modeling_realm:         atmos
   

In [52]:
# Make the forcing files from the inputs given
make_forcing.make_cfsite_forcing(verbose=True,ds=ds ) # do w/o dropping sites

In [85]:
ds['zg']

<xarray.DataArray 'zg' (time: 144000, site: 119, lev: 38)>
dask.array<concatenate, shape=(144000, 119, 38), dtype=float32, chunksize=(17280, 119, 38), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2000-09-01 00:00:00 ... 2008-12-30 23:30:00
  * site     (site) int32 1 2 3 4 5 6 7 8 9 ... 112 113 114 115 116 117 118 119
  * lev      (lev) float64 20.0 80.0 180.0 ... 2.922e+04 3.291e+04 3.925e+04
    lat      (site) float32 dask.array<chunksize=(119,), meta=np.ndarray>
    lon      (site) float32 dask.array<chunksize=(119,), meta=np.ndarray>
Attributes:
    standard_name:     geopotential_height
    long_name:         Geopotential Height
    units:             m
    original_name:     mo: m01s16i201
    cell_methods:      time: point
    history:           2011-12-16T14:57:19Z altered by CMOR: replaced missing...
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

In [13]:
# do selecting sites below 45 S
reload(make_forcing)
make_forcing.make_cfsite_forcing(verbose=True,ds=ds,sites = ds['site'][ds['lat']  < -45].values, out_path = '/export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/southern_ocean_forcing_test/')

/export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/python/make_forcing.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (sites == 'all') or (sites is None): sites   = ds.site.values # all sites


processing: 2000-01
modifying forcing
skipping saving because data slice is empty: /export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/southern_ocean_forcing_test//HadGEM2-A/amip//HadGEM2-A_amip_r1i1p1.2000-01.nc
processing: 2000-02
modifying forcing
skipping saving because data slice is empty: /export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/southern_ocean_forcing_test//HadGEM2-A/amip//HadGEM2-A_amip_r1i1p1.2000-02.nc
processing: 2000-03
modifying forcing
skipping saving because data slice is empty: /export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/southern_ocean_forcing_test//HadGEM2-A/amip//HadGEM2-A_amip_r1i1p1.2000-03.nc
processing: 2000-04
modifying forcing
skipping saving because data slice is empty: /export/data1/jbenjami/Research_Schneider/Data/CMIP5/CFMIP2/southern_ocean_forcing_test//HadGEM2-A/amip//HadGEM2-A_amip_r1i1p1.2000-04.nc
processing: 2000-05
modifying forcing
skipping saving because data slice is empty: /export/data1/jbenjami/Research_S

KeyboardInterrupt: 